In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, EarlyStoppingCallback
from transformers import DataCollatorForSeq2Seq
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset
from datasets import concatenate_datasets
from transformers import T5ForConditionalGeneration, T5Tokenizer
from random import randrange
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
from collections import Counter
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [1]:
!pip install nltk

In [2]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 15.4 MB/s eta 0:00:00


In [3]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24936 sha256=304438dc8e38612f554bb6476737726409622240796281449cc6e5ecfade2325
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge_score


In [4]:
!pip install py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.0/93.0 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 15.4 MB/s eta 0:00:00


In [5]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 3.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 13.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 62.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 13.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 17.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 16.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
!pip install transformers --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 71.6 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 99.6 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.20.1
    Uninstalling transformers-4.20.1:
      Successfully uninstalled transformers-4.20.1


source: https://towardsdatascience.com/how-to-adapt-a-multilingual-t5-model-for-a-single-language-b9f94f3d9c90

## model

In [3]:
model_id="google/mt5-small"
# model_id="mt5_et/" #this has vocab adjusted
tokenizer = T5Tokenizer.from_pretrained(model_id)
model = T5ForConditionalGeneration.from_pretrained(model_id)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


In [4]:
def msize(m):
    return sum(p.numel() for p in m.parameters())
print(msize(model.shared) / msize(model))   
print(msize(model.lm_head) / msize(model))  

0.4266064454395085
0.4266064454395085


about 42% are embeddings

## data

In [2]:
dataset_id = "TalTechNLP/samsum_ee"
# Load dataset from the hub
dataset = load_dataset(dataset_id)

print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

Using custom data configuration TalTechNLP--samsum_ee-c4fb6b631b04898b
Reusing dataset parquet (/root/.cache/huggingface/datasets/TalTechNLP___parquet/TalTechNLP--samsum_ee-c4fb6b631b04898b/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8)


  0%|          | 0/3 [00:00<?, ?it/s]

Train dataset size: 14732
Test dataset size: 819


In [3]:
train_dataset = dataset['train'].filter(lambda example, idx: example['summary'] is not None and example['dialogue'] is not None, with_indices=True)
train_dataset

Loading cached processed dataset at /root/.cache/huggingface/datasets/TalTechNLP___parquet/TalTechNLP--samsum_ee-c4fb6b631b04898b/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8/cache-e71b8d41c6d11842.arrow


Dataset({
    features: ['id', 'dialogue', 'summary', 'en_dialogue', 'en_summary'],
    num_rows: 13199
})

In [4]:
test_dataset = dataset['test'].filter(lambda example, idx: example['summary'] is not None and example['dialogue'] is not None, with_indices=True)
test_dataset

Loading cached processed dataset at /root/.cache/huggingface/datasets/TalTechNLP___parquet/TalTechNLP--samsum_ee-c4fb6b631b04898b/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8/cache-ba4f45479de509c8.arrow


Dataset({
    features: ['id', 'dialogue', 'summary', 'en_dialogue', 'en_summary'],
    num_rows: 809
})

## update model vocabulary

In [8]:
texts4vocab=train_dataset['dialogue']+train_dataset['summary']
len(texts4vocab)

26398

In [9]:
cnt_et = Counter()
for text in texts4vocab:
    cnt_et.update(tokenizer.encode(text))
print(len(cnt_et), len(cnt_et)/tokenizer.vocab_size)  

22815 0.0912235105957617


In [10]:
cnt_et.most_common(20)

[(259, 222378),
 (267, 152951),
 (260, 109486),
 (261, 103717),
 (351, 37623),
 (291, 35910),
 (262, 29563),
 (1, 26398),
 (263, 24319),
 (432, 24040),
 (265, 23799),
 (309, 21963),
 (266, 21379),
 (383, 20708),
 (327, 19689),
 (316, 18386),
 (1055, 17737),
 (496, 17714),
 (1221, 15946),
 (178005, 14920)]

In [11]:
new_tokens = set(range(1000))
for i, (k, v) in enumerate(cnt_et.items()):
    if k not in new_tokens:
        new_tokens.add(k)
for t in range(tokenizer.vocab_size - 200, tokenizer.vocab_size):
    new_tokens.add(t)
print(len(new_tokens))
kept_ids = sorted(new_tokens)

23522


### update model

In [12]:
new_size = len(kept_ids)
new_emb = torch.nn.Embedding(new_size, model.shared.embedding_dim)
new_head = torch.nn.Linear(in_features=model.lm_head.in_features, out_features=new_size, bias=False)
for new_id, old_id in enumerate(kept_ids):
    new_emb.weight.data[new_id] = model.shared.weight.data[old_id]
    new_head.weight.data[new_id] = model.lm_head.weight.data[old_id]
model.shared.weight = new_emb.weight
model.lm_head.weight = new_head.weight
model.config.__dict__['vocab_size'] = new_size
model.config.__dict__['_name_or_path'] = 'mt5_et/mt_et_t5-small'

### update tokenizer

In [1]:
!apt-get update 
!apt install protobuf-compiler -y

Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:2 https://deb.nodesource.com/node_16.x focal InRelease                     
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease                         
Hit:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease             
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libprotobuf-dev libprotobuf-lite17 libprotobuf17 libprotoc17
The following NEW packages will be installed:
  libprotobuf-dev libprotobuf-lite17 libprotobuf17 libprotoc17
  protobuf-compiler
0 upgraded, 5 newly installed, 0 to remove and 212 not upgraded.
Need to get 2760 kB of archives.
After this operation, 16.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu

In [13]:
! wget https://raw.githubusercontent.com/google/sentencepiece/master/src/sentencepiece_model.proto
! protoc --python_out=. sentencepiece_model.proto
import sentencepiece.sentencepiece_model_pb2 as spmp
smp = tokenizer.sp_model.serialized_model_proto()
m = spmp.ModelProto()
m.ParseFromString(smp)
print('the loaded model has pieces:', len(m.pieces))
new_pieces = [m.pieces[idx] for idx in kept_ids]
print('the new pieces:', len(new_pieces))
# replace the content of the first 30K pieces
for i, p in enumerate(new_pieces):
    m.pieces[i].piece = p.piece
    m.pieces[i].score = p.score
    m.pieces[i].type = p.type
# drop the remaining pieces
n = len(new_pieces)
for i in range(len(m.pieces) - n):
    m.pieces.pop(len(m.pieces) - 1)
print(len(m.pieces))
with open('mt5_et/new_sp.model', 'wb') as f:
    f.write(m.SerializeToString())
new_tokenizer = T5Tokenizer('mt5_et/new_sp.model', extra_ids=0)

--2024-04-08 07:05:18--  https://raw.githubusercontent.com/google/sentencepiece/master/src/sentencepiece_model.proto
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14023 (14K) [text/plain]
Saving to: ‘sentencepiece_model.proto.5’

sentencepiece_model 100%[===================>]  13.69K  --.-KB/s    in 0s      

2024-04-08 07:05:18 (70.1 MB/s) - ‘sentencepiece_model.proto.5’ saved [14023/14023]

the loaded model has pieces: 250100
the new pieces: 23522
23522


In [14]:
new_tokenizer.save_pretrained('mt5_et')
model.save_pretrained('mt5_et')

In [15]:
new_tokenizer('tere tulemast!')

{'input_ids': [767, 265, 3123, 11216, 309, 1], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [5]:
tokenizer = T5Tokenizer.from_pretrained('mt5_et')
model = T5ForConditionalGeneration.from_pretrained('mt5_et')

## prep data for training

In [6]:
# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([train_dataset, test_dataset]).map(lambda x: tokenizer(x["dialogue"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
max_source_length=1024
print(f"Max source length: {max_source_length}")

Parameter 'function'=<function <lambda> at 0x7fdbdc72f9d0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/15 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Max source length: 1024


In [7]:
# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([train_dataset, test_dataset]).map(lambda x: tokenizer(x["summary"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
max_target_length=512
print(f"Max target length: {max_target_length}")

  0%|          | 0/15 [00:00<?, ?ba/s]

Max target length: 512


In [8]:
padding="max_length"
sample = dataset['train'][randrange(len(dataset["train"]))]
inputs = ["summarize: " + item for item in sample["dialogue"]]

In [9]:
model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)
labels = tokenizer(text=sample["summary"], max_length=max_target_length, padding=padding, truncation=True)
model_inputs["labels"] = labels["input_ids"]

In [10]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = ["summarize: " + item for item in sample["dialogue"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text=sample["summary"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=["dialogue", "summary", "id"])
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True, remove_columns=["dialogue", "summary", "id"])

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [11]:
# tokenized_train_dataset[0]

## evaluator

In [12]:
# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

In [13]:
# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

## training 

### try peft

https://huggingface.co/blog/peft

In [14]:
from peft import get_peft_model, LoraConfig, TaskType

In [15]:
# peft_config = LoraConfig(
#     task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
# )
# model = get_peft_model(model, peft_config)
# model.print_trainable_parameters()

In [16]:
early_stop = EarlyStoppingCallback(2, 0.0)

In [22]:
# model_id="google/flan-t5-base"
# Hugging Face repository id
model_id="google/mt5-small"
repository_id = f"{model_id.split('/')[1]}-{dataset_id}"#for some reason this was not working
# repository_id="mt5-small-TalTechNLP/samsum_ee"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=20,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=False,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
    #generation_max_length=128
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[early_stop]
)

/usr/local/lib/python3.9/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
# Start training
trainer.train(resume_from_checkpoint=False)

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.627000,2.624735,29.255000,10.187800,25.898200,27.771700,17.782447
2,2.963300,2.437816,29.630700,10.388800,26.030600,27.952400,17.096415
3,2.833000,2.376554,29.734200,10.805100,26.296100,28.113100,17.296663
4,2.733300,2.342344,29.442900,10.687100,26.207100,27.922800,17.456119
5,2.648600,2.306559,30.343000,11.368400,26.658300,28.681600,16.861557
6,2.612400,2.287098,30.594800,11.333100,26.848900,28.829600,17.367120
7,2.549800,2.256288,30.886700,11.811000,27.205000,29.139400,17.229913
8,2.497100,2.243558,30.782800,11.613400,27.161000,29.053900,17.156984
9,2.458900,2.228512,31.527600,11.874600,27.444200,29.558800,16.903585


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the

## example usage

In [11]:
from peft import PeftModel, PeftConfig

In [28]:
peft_model_id='mt5-small-TalTechNLP/samsum_ee/checkpoint-33000/'
# config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(peft_model_id)
# model = PeftModel.from_pretrained(model, peft_model_id)
# tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

In [43]:
# model.

In [29]:
model=model.to('cuda')

In [45]:
input_text = dataset['test'][0]['dialogue']
input_ids = tokenizer(
     input_text, return_tensors="pt"
).input_ids  # Batch size 1
outputs = model.generate(input_ids=input_ids.to('cuda'), max_new_tokens=512)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Amanda saadab Betty telefoninumbri. Betty helistas talle viimati, kui nad koos pargis oli. Larry helistas talle viimati, kui nad koos pargis oli.


In [31]:
input_ids = tokenizer(
    "summarize: studies have shown that owning a dog is good for you", return_tensors="pt"
).input_ids  # Batch size 1
outputs = model.generate(input_ids=input_ids.to('cuda'))
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

studies on shownud, et oma dog on korras.


In [32]:
input_text

'Hannah: Hei, kas sul on Betty number?\nAmanda: Vaatan järele.\nHannah: <file_gif>\nAmanda: Vabandust, ei leia seda.\nAmanda: Küsi Larrylt.\nAmanda: Ta helistas talle viimati, kui me koos pargis olime.\nHannah: Ma ei tunne teda hästi.\nHannah: <file_gif>\nAmanda: Ära ole häbelik, ta on väga tore.\nHannah: Kui sa ütled nii..\nHannah: Ma eelistaksin, et sa talle sõnumi saadaksid.\nAmanda: Lihtsalt saada talle sõnum 🙂\nHannah: Urgh.. Olgu siis\nHannah: Nägemist\nAmanda: Nägemist-nägemist'

In [33]:
input_ids.shape

torch.Size([1, 21])

In [34]:
input_text = dataset['test'][1]['dialogue']
input_ids = tokenizer(
     input_text, return_tensors="pt"
).input_ids  # Batch size 1
outputs = model.generate(input_ids=input_ids.to('cuda'))
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Eric vaatab rongi osa. Rob vaatab rongi osa Youtube'is.


In [35]:
input_text

"Eric: MASIN!\nRob: See on nii lahe!\nEric: Ma tean! Ja näitab, kuidas ameeriklased näevad venelasi ;)\nRob: Ja see on tõesti naljakas!\nEric: Ma tean! Mulle meeldib eriti rongi osa!\nRob: Hahaha! Keegi ei räägi masinaga nii!\nEric: Kas see on tema ainus etendus?\nRob: Ei tea. Ma vaatan järele.\nEric: Kindlasti.\nRob: Selgub, et ei! Mõned tema etendused on Youtube'is.\nEric: Lahe! Ma vaatan neid kohe!\nRob: Mina ka!\nEric: MASIN!\nRob: MASIN!\nEric: Kuni kohtumiseni?\nRob: Muidugi :)"

In [36]:
dataset['test'][1]['summary']

"Eric ja Rob lähevad vaatama stand-up'i YouTube'is."

In [37]:
input_text = dataset['test'][2]['dialogue']
input_ids = tokenizer(
     input_text, return_tensors="pt"
).input_ids  # Batch size 1
outputs = model.generate(input_ids=input_ids.to('cuda'))
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Lenny soovib, et Lenny ostaks esimese või kolmanda paari. Lenny


In [38]:
input_text

'Lenny: Kallis, kas sa saaksid mulle millegagi abi anda?\nBob: Muidugi, mis toimub?\nLenny: Millise ma peaksin valima?\nBob: Saada mulle pilte.\nLenny: <file_photo>\nLenny: <file_photo>\nLenny: <file_photo>\nBob: Mulle meeldivad kõige rohkem esimesed.\nLenny: Aga mul on juba lillad püksid. Kas on mõtet omada kahte paari?\nBob: Mul on neli musta paari :D :D\nLenny: Jah, aga kas ma ei peaks valima erinevat värvi?\nBob: Oluline on see, milline annab sulle kõige rohkem riietumisvõimalusi.\nLenny: Nii et ma arvan, et ostan esimese või kolmanda paari siis.\nBob: Vali parim kvaliteet siis.\nLenny: Sul on õigus, aitäh.\nBob: Pole probleemi :)'

In [39]:
dataset['test'][2]['summary']

'Lenny ei suuda otsustada, millised püksid osta. Bob nõustas Lennyt selles küsimuses. Lenny läheb Bobi nõu järgi ja valib kvaliteetseimad püksid.'